In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:

chunk_size = 100000
op_path = 'output-single'
ngroup = 10


In [3]:

def get_stat(ret_df, max_lag: int = None):

    inner_df = ret_df.copy()

    ret_mean = inner_df.mean() * 100

    if max_lag == None:
        ret_t = stats.ttest_1samp(inner_df, 0)[0]
        ret_p = stats.ttest_1samp(inner_df, 0)[1]
        ret_t = pd.Series(ret_t, index=ret_mean.index)
        ret_p = pd.Series(ret_p, index=ret_mean.index)
    else:
        assert type(max_lag) == int, "input an integer max_lag"
        ret_t = []
        ret_p = []
        for col in inner_df.columns:
            reg = smf.ols(f"{col} ~ 1", data=inner_df).fit(
                cov_type='HAC', cov_kwds={'maxlags': max_lag})
            t_v = reg.tvalues['Intercept']
            p_v = reg.pvalues['Intercept']
            ret_t.append(t_v)
            ret_p.append(p_v)
        ret_t = pd.Series(ret_t, index=ret_mean.index)
        ret_p = pd.Series(ret_p, index=ret_mean.index)
    
    ret_mean.name = 'mean'
    ret_t.name = 't'
    ret_p.name = 'p'
    
    stats_data = pd.DataFrame([ret_mean, ret_t, ret_p])
    return stats_data


def read_csv(csv_path, chunk_size=100000):
    try:
        # 方法 A: 标准读取 (默认逗号分隔)
        #df_csv = pd.read_csv('CHN24/all_predictors.csv')
        # 强制将 'STKCD' 列读取为字符串，保留 000002
        df_csv = pd.read_csv(csv_path, dtype={'STKCD': str})

        # 检查一下
        print(df_csv['STKCD'].head())
    except UnicodeDecodeError:
        # 方法 B: 如果是中文 CSV (特别是 Excel 导出的)，通常需要 gbk 或 gb18030 编码
        print("默认编码失败，尝试 GBK...")
        df_csv = pd.read_csv('CHN24/all_predictors.csv', encoding='gbk')

    # 预览 CSV 数据
    print("\nCSV 数据预览：")
    print(df_csv.head())
    return df_csv

def match_df_flex(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    *,
    left_on: list[str],
    right_on: list[str],
    how: str = "left",
    validate: str = "many_to_one"
):
    if len(left_on) != len(right_on):
        raise ValueError("left_on 和 right_on 长度必须一致")

    for col in left_on:
        if col not in df1.columns:
            raise ValueError(f"[df1] 缺少列: {col}")

    for col in right_on:
        if col not in df2.columns:
            raise ValueError(f"[df2] 缺少列: {col}")

    # df2 去重保护
    if df2.duplicated(subset=right_on).any():
        raise ValueError("df2 在 right_on 上存在重复键")

    df_merged = pd.merge(
        df1,
        df2,
        how=how,
        left_on=left_on,
        right_on=right_on,
        validate=validate
    )

    return df_merged


def cleanBlank(df, sort1, sort2):
    # 1️⃣ 先保存一份 df
    df = df.copy()

    # 2️⃣ 排序
    df = df.sort_values(by=[sort1, sort2])

    # 3️⃣ 记录删除前行数
    n_before = len(df)

    # 4️⃣ 删除含 NaN 的行（只要有一个 NaN 就删）
    df = df.dropna(axis=0)

    # 5️⃣ 记录删除后行数
    n_after = len(df)

    # 6️⃣ 打印删除信息
    print(f"firstSort: 删除了 {n_before - n_after} 行（{n_before} → {n_after}）")

    return df


# ---------- 工具：任意格式 -> YYYYMM(Int64) ----------
def to_yyyymm(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip()

    # 情况A：已经是 6 位 YYYYMM
    mask6 = s.str.fullmatch(r"\d{6}", na=False)

    out = pd.Series([pd.NA] * len(s), index=s.index, dtype="Int64")

    # 6位直接转
    out.loc[mask6] = s.loc[mask6].astype("Int64")

    # 情况B：YYYY-MM / YYYY-MM-DD / datetime 等
    dt = pd.to_datetime(s.loc[~mask6], errors="coerce")
    out.loc[~mask6] = (dt.dt.year * 100 + dt.dt.month).astype("Int64")

    return out


In [ ]:
df_csv=pd.read_csv('source/ghz_factors.csv', dtype={'STKCD': str})

In [ ]:
print(df_csv.columns.tolist())
print(df_csv.head())
df_csv['permno'] = pd.to_numeric(df_csv['permno'], errors='coerce').astype('Int64')
print(df_csv['permno'])

In [ ]:

# ---------- 统一 TRDMNT 为 YYYYMM ----------
df_csv["TRDMNT"] = to_yyyymm(df_csv["DATE"])

print("TRDMNT dtype:", df_csv["TRDMNT"].dtype)
print("TRDMNT NA:", df_csv["TRDMNT"].isna().sum())
print(df_csv[["TRDMNT"]].head())

# ---------- 定义起止时间（YYYYMM int） ----------
start_month = 200001
end_month   = 202412
# 日期筛选
df_csv_filtered = df_csv.loc[
    (df_csv["TRDMNT"] >= start_month) & (df_csv["TRDMNT"] <= end_month)
].copy()

print("before/after:", len(df_csv), len(df_csv_filtered))
print("min/max after:", df_csv_filtered["TRDMNT"].min(), df_csv_filtered["TRDMNT"].max())
df_csv=df_csv_filtered

In [ ]:
os.makedirs(op_path, exist_ok=True)
df_csv_filtered.to_csv(os.path.join(op_path, "us_single.csv"), index=False)


# 因子处理

In [4]:
df_csv_all=pd.read_csv('output-single/us_single.csv', dtype={'STKCD': str})

In [17]:
non_factor_cols = [
    # 1️⃣ ID / 身份标识（纯索引）
    'permno',      # CRSP 股票唯一标识
    'gvkey',       # Compustat 公司唯一标识
    # 2️⃣ 时间 / 事件索引（不是特征）
    'fyear',       # 财年
    'DATE',        # 交易日期
    'datadate',    # 会计数据日期
    'rdq',         # 财报披露日期
    'eamonth',     # 财报月份索引
    # 3️⃣ 市场制度 / 状态标签
    'exchcd',      # 交易所代码
    'IPO',         # IPO 状态标记
    # 4️⃣ 原始价格 / 规模 / 交易量（因子“原料”，不是因子）
    'prc',         # 股价
    'prccq',       # 季度收盘价
    'SHROUT',      # 流通股数
    'VOL',         # 成交量
    'dolvol',      # 成交额
    'mve',         # 市值（原始）
    'mve_m',       # 月度市值
    'pps',         # 每股价格
    # 5️⃣ 收益 & 退市相关（被解释变量 / 状态变量）
    'RET',         # 股票收益率（因变量）
    'dlret',       # 退市收益
    'dlstcd',      # 退市代码
    # 6️⃣ 回测 / 分组 / 统计产物（严禁当因子）
    'count',       # 分组样本数
    'ewret',       # 等权组合收益
    # 7️⃣ 临时索引 / 垃圾列（必须清理）
    'i',           # 中间索引列
    'j',    'TRDMNT'       # 中间索引列
]


# 只保留真正的因子列
factor_cols = [
    c for c in df_csv_all.columns
    if c not in non_factor_cols
]

print("因子列数量:", len(factor_cols))
print("前10个因子列:", factor_cols[:10])


因子列数量: 126
前10个因子列: ['sic2', 'spi', 'mve_f', 'bm', 'ep', 'cashpr', 'dy', 'lev', 'sp', 'roic']


# 选取小表
任意选取了一个指标,如“RDM”

In [ ]:
success_factors = []
failed_factors = []

l = len(factor_cols)

for i in range(l):
    sort_factor = factor_cols[i]

    # 只取必要列
    sub_table_test = df_csv_all[['permno', 'TRDMNT', sort_factor, 'RET', 'mve_f']]

    # 清洗
    df_clean_test = cleanBlank(sub_table_test, 'TRDMNT', 'permno')

    try:
        sub_table_groupped_test = GroupN(
            df_clean_test,
            sort_var='TRDMNT',
            vars=sort_factor,
            n_group=ngroup
        )
        success_factors.append(sort_factor)

    except Exception as e:
        failed_factors.append({
            "factor": sort_factor,
            "error": str(e)
        })
        continue

print("成功分组因子数量:", len(success_factors))
print("失败分组因子数量:", len(failed_factors))

firstSort: 删除了 0 行（1188139 → 1188139）
firstSort: 删除了 63032 行（1188139 → 1125107）
firstSort: 删除了 0 行（1188139 → 1188139）
firstSort: 删除了 0 行（1188139 → 1188139）
firstSort: 删除了 0 行（1188139 → 1188139）
firstSort: 删除了 7822 行（1188139 → 1180317）
firstSort: 删除了 1830 行（1188139 → 1186309）
firstSort: 删除了 2701 行（1188139 → 1185438）
firstSort: 删除了 0 行（1188139 → 1188139）
firstSort: 删除了 10525 行（1188139 → 1177614）
firstSort: 删除了 590609 行（1188139 → 597530）
firstSort: 删除了 564223 行（1188139 → 623916）
firstSort: 删除了 50110 行（1188139 → 1138029）
firstSort: 删除了 50110 行（1188139 → 1138029）
firstSort: 删除了 50556 行（1188139 → 1137583）
firstSort: 删除了 53723 行（1188139 → 1134416）
firstSort: 删除了 90097 行（1188139 → 1098042）
firstSort: 删除了 90097 行（1188139 → 1098042）
firstSort: 删除了 42246 行（1188139 → 1145893）
firstSort: 删除了 90097 行（1188139 → 1098042）
firstSort: 删除了 0 行（1188139 → 1188139）
firstSort: 删除了 64888 行（1188139 → 1123251）
firstSort: 删除了 0 行（1188139 → 1188139）
firstSort: 删除了 90097 行（1188139 → 1098042）
firstSort: 删除了 52153 行（

In [ ]:
sort_factor = success_factors[0]
sub_table = df_csv_all[['permno','TRDMNT',sort_factor,'RET','mve_f']]

In [26]:
df_clean=cleanBlank(sub_table, 'TRDMNT','permno')
print(df_clean.head())

firstSort: 删除了 0 行（1188139 → 1188139）
     permno  TRDMNT        bm       RET       mve_f
0     10001  200001  0.644588 -0.044118   20.993250
207   10002  200001  0.509429 -0.025641  115.710000
365   10009  200001  0.757597 -0.008475   50.610000
376   10012  200001  0.130151 -0.097276   34.628937
444   10016  200001  0.183991 -0.099338  300.372813


In [27]:
#========================================================
#                   第一步、分组
#========================================================

def GroupN(in_df, sort_var, vars, n_group=10):
    out_df = in_df.copy()
    out_df[f"{vars}_g{n_group}"] = out_df.groupby(sort_var)[vars].transform(
        lambda x: pd.qcut(x, q=n_group, labels=[i for i in range(1, n_group+1)]))
    out_df[f"{vars}_g{n_group}"] = out_df[f"{vars}_g{n_group}"] .astype(int)
    return out_df


sub_table_groupped = GroupN(df_clean, 'TRDMNT', sort_factor, n_group = ngroup)

In [28]:
#========================================================
#                   第二步、缩尾处理
#          对除了keep_cols以外的所有因子进行缩尾处理
#========================================================

class Winsorize:
    def __init__(self, in_df, sort_var, vars, perc=1, trim=0) -> None:
        self.in_df = in_df
        self.sort_var = sort_var
        self.vars = vars
        self.perc = perc
        self.trim = trim

    def func_trim(self, in_ser, perc):
        perc_upper = (100 - perc) / 100
        perc_lower = perc / 100

        qt_lower, qt_upper = in_ser.quantile([perc_lower, perc_upper])
        in_ser[in_ser > qt_upper] = np.nan
        in_ser[in_ser < qt_lower] = np.nan
        return in_ser

    def func_winsor(self, in_ser, perc):
        perc_upper = (100 - perc) / 100
        perc_lower = perc / 100
        qt_lower, qt_upper = in_ser.quantile([perc_lower, perc_upper])

        in_ser[in_ser > qt_upper] = qt_upper
        in_ser[in_ser < qt_lower] = qt_lower
        return in_ser

    def get(self, ):
        out_df = self.in_df.copy()
        if self.trim == 1:
            proc_method = self.func_trim
        if self.trim == 0:
            proc_method = self.func_winsor

        out_df[f"{self.vars}"] = out_df.groupby(
            self.sort_var)[self.vars].transform(lambda x: proc_method(x, 1))

        return out_df

In [29]:
#Winsorize
#先对size进行缩尾处理
winsor = Winsorize(sub_table_groupped, "TRDMNT",'mve_f')
sub_table_groupped = winsor.get()
# 第二次缩尾：按（月 × 因子组）对 size 缩尾
winsor = Winsorize(sub_table_groupped, ["TRDMNT",f"{sort_factor}_g{ngroup}" ],'mve_f')
sub_table_groupped = winsor.get()

In [31]:

#========================================================
#                   第三步、计算EW VW
#           EW=同一个月 (TRDMNT)
#           同一个因子分组（比如 AM_g10 的第 3 组）
#           把这一组里所有股票的收益率 ret 简单平均。
#           VW=同一个月
#           同一个因子组
#           用上一期市值 size 当权重，对收益率加权平均
#
#       在这里的收益率指的是RET [考虑现金红利再投资的月个股回报率]
#       从这里开始就只有一个了，上面都是对全部的因子进行循环处理
#=======================================================

import statsmodels.formula.api as smf

def get_stat(ret_df, max_lag: int = None):

    inner_df = ret_df.copy()

    ret_mean = inner_df.mean() * 100

    if max_lag == None:
        ret_t = stats.ttest_1samp(inner_df, 0)[0]
        ret_p = stats.ttest_1samp(inner_df, 0)[1]
        ret_t = pd.Series(ret_t, index=ret_mean.index)
        ret_p = pd.Series(ret_p, index=ret_mean.index)
    else:
        assert type(max_lag) == int, "input an integer max_lag"
        ret_t = []
        ret_p = []
        for col in inner_df.columns:
            reg = smf.ols(f"{col} ~ 1", data=inner_df).fit(
                cov_type='HAC', cov_kwds={'maxlags': max_lag})
            t_v = reg.tvalues['Intercept']
            p_v = reg.pvalues['Intercept']
            ret_t.append(t_v)
            ret_p.append(p_v)
        ret_t = pd.Series(ret_t, index=ret_mean.index)
        ret_p = pd.Series(ret_p, index=ret_mean.index)
    
    ret_mean.name = 'mean'
    ret_t.name = 't'
    ret_p.name = 'p'
    
    stats_data = pd.DataFrame([ret_mean, ret_t, ret_p])
    return stats_data

in_ret = sub_table_groupped.copy(deep =True)
print(in_ret.head(5))

ew_ret = in_ret.groupby(['TRDMNT', f"{sort_factor}_g{ngroup}"])['RET'].mean()
vw_ret = (
    in_ret.groupby(['TRDMNT', f"{sort_factor}_g{ngroup}"])
          .apply(lambda g: np.average(g['RET'], weights=g['mve_f']))
)
vw_ret.name = "Vw_ret"

ew_mean = ew_ret.copy(deep=True)
ew_mean.name = 'Ew_ret'

vw_mean = vw_ret.copy(deep=True)
vw_mean.name = 'Vw_ret'

month_count = in_ret.groupby(
    ['TRDMNT', f"{sort_factor}_g{ngroup}"]
)['RET'].count()
month_count.name = 'Count'

sort_factor_mean = in_ret.groupby(
    ['TRDMNT', f"{sort_factor}_g{ngroup}"]
)[sort_factor].mean()

month_result = pd.concat(
    [month_count, sort_factor_mean, ew_mean, vw_mean],
    axis=1,
    ignore_index=False
)

ew_ret = ew_ret.unstack()
vw_ret = vw_ret.unstack()

ew_ret.columns = [f"col_{i+1}" for i in range(ngroup)]
vw_ret.columns = [f"col_{i+1}" for i in range(ngroup)]

ew_ret['high_low'] = ew_ret[f"col_{ngroup}"] - ew_ret["col_1"]
vw_ret['high_low'] = vw_ret[f"col_{ngroup}"] - vw_ret["col_1"]

ew_other = ew_ret.loc[:, ['high_low']]
ew_other = ew_other.stack()
ew_other.name = 'Ew_ret'

vw_other = vw_ret.loc[:, ['high_low']]
vw_other = vw_other.stack()
vw_other.name = 'Vw_ret'

other = pd.concat([ew_other, vw_other], axis=1, ignore_index=False)
other = other.reset_index()
other = other.rename(columns={'level_1': f"{sort_factor}_g{ngroup}"})
other = other.set_index(['TRDMNT', f"{sort_factor}_g{ngroup}"])

month_result = pd.concat([month_result, other], axis=0, ignore_index=False)
month_result.sort_index(inplace=True)

month_result.to_csv(os.path.join(op_path, f"{sort_factor}_month_result.csv"))

# 如果 index 是 PeriodIndex，用 to_timestamp
if isinstance(ew_ret.index, pd.PeriodIndex):
    ew_ret.index = ew_ret.index.to_timestamp(how="end")

if isinstance(vw_ret.index, pd.PeriodIndex):
    vw_ret.index = vw_ret.index.to_timestamp(how="end")

ew_stat = get_stat(ew_ret, max_lag = 3)
vw_stat = get_stat(vw_ret, max_lag = 3)

ew_stat.to_csv(os.path.join(op_path, f"{sort_factor}_ew_result.csv"))
vw_stat.to_csv(os.path.join(op_path, f"{sort_factor}_vw_result.csv"))


     permno  TRDMNT        bm       RET       mve_f  bm_g10
0     10001  200001  0.644588 -0.044118   20.993250       7
207   10002  200001  0.509429 -0.025641  115.710000       5
365   10009  200001  0.757597 -0.008475   50.610000       7
376   10012  200001  0.130151 -0.097276   34.628937       2
444   10016  200001  0.183991 -0.099338  300.372813       2
